# Data Engineering for Auxiliary Data

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
from pathlib import Path

import pandas as pd
import numpy as np

from geopy import distance
from src.preprocessing import update_data

In [3]:
def get_nearest_y(X, y):
    """
    input:
        X - a list of tuples representing (lat, lng)
        y - a list of tuples representing (lat, lng)
    return:
        distance_to_nearest - the distance from X to the closest y (in meters)
        index_of_nearest - the index of the closest y from X
    """
    distance_to_nearest = []
    index_of_nearest = []

    for point_x in X:
        distance_to_y = []

        for point_y in y:
            distance_to_y.append(distance.distance(point_x, point_y).meters)

        distance_to_y = np.array(distance_to_y)
        min_distance = np.min(distance_to_y, axis=0)
        min_index = np.argmin(distance_to_y, axis=0)

        distance_to_nearest.append(min_distance)
        index_of_nearest.append(min_index)

    return distance_to_nearest, index_of_nearest


def get_nearest_distances(df_X, df_y):

    # Zip the lat and lng into tuples
    X_coords = df_X[['lat', 'lng']].apply(tuple, axis=1)
    y_coords = df_y[['lat', 'lng']].apply(tuple, axis=1)

    distance_result, index_result = get_nearest_y(X_coords, y_coords)
    return distance_result, index_result


In [4]:
data_dir = Path("./raw_data/")
aux_data_dir = Path("./raw_data/auxiliary-data/")

In [5]:
def generate_auxiliary_data(df):
    
    df_aux = pd.DataFrame(index=df.index)
    df_aux['lat'] = df['lat']
    df_aux['lng'] = df['lng']

    # MRT
    df_sg_mrt_stations = pd.read_csv(Path(aux_data_dir, "sg-mrt-stations.csv"))
    distance_mrt, index_mrt = get_nearest_distances(df_aux, df_sg_mrt_stations)
    df_aux['dist_meters_to_nearest_mrt'] = distance_mrt
    df_aux['nearest_mrt_code'] = list(df_sg_mrt_stations.iloc[index_mrt]['code'])
    df_aux['nearest_mrt_line'] = list(df_sg_mrt_stations.iloc[index_mrt]['line'])

    # Shopping mall
    df_sg_shopping_malls = pd.read_csv(Path(aux_data_dir, "sg-shopping-malls.csv"))
    distance_mall, index_mall = get_nearest_distances(df_aux, df_sg_shopping_malls)
    df_aux['dist_meters_to_nearest_mall'] = distance_mall
    df_aux['nearest_mall_index'] = index_mall

    # Commercial centres
    df_sg_commercial_centres = pd.read_csv(Path(aux_data_dir, "sg-commerical-centres.csv"))
    distance_commercial, index_commercial = get_nearest_distances(df_aux, df_sg_commercial_centres)
    df_aux['dist_meters_to_nearest_commercial_centre'] = distance_commercial
    df_aux['nearest_commercial_centre_index'] = index_commercial

    # Primary schools
    df_sg_pri_sch = pd.read_csv(Path(aux_data_dir, "sg-primary-schools.csv"))
    distance_pri_sch, index_pri_sch = get_nearest_distances(df_aux, df_sg_pri_sch)
    df_aux['dist_meters_to_nearest_primary_school'] = distance_pri_sch
    df_aux['nearest_primary_school_index'] = index_pri_sch

    # Secondary schools
    df_sg_sec_sch = pd.read_csv(Path(aux_data_dir, "sg-secondary-schools.csv"))
    distance_sec_sch, index_sec_sch = get_nearest_distances(df_aux, df_sg_sec_sch)
    df_aux['dist_meters_to_nearest_secondary_school'] = distance_sec_sch
    df_aux['nearest_secondary_school_index'] = index_sec_sch

    return df_aux

## Generate the auxiliary features for nearest community facilities

This takes around 8 seconds per 100 records.

For the entire training and test set, it takes around 40 minutes (_Probably have some room to optimize this_)

In [6]:
df_train = pd.read_csv(Path(data_dir, "train.csv"))
df_test = pd.read_csv(Path(data_dir, "test.csv"))

# Clean the records with dirty lat, lng identified in EDA
update_data(df_train)
update_data(df_test)

,listing_id,title,address,property_name,property_type,tenure,built_year,num_beds,num_baths,size_sqft,floor_level,furnishing,available_unit_types,total_num_units,property_details_url,lat,lng,elevation,subzone,planning_area
0,777912,1 bed condo for sale in the gazania,17 how sun drive,the gazania,condo,freehold,2022.0,1.0,1.0,463,NaN,unfurnished,"studio, 1, 2, 3, 4, 5 br",250.0,https://www.99.co/singapore/condos-apartments/...,1.344334,103.878690,0,upper paya lebar,serangoon
1,936612,3 bed condo for sale in vue 8 residence,95 pasir ris heights,vue 8 residence,Condo,99-year leasehold,2017.0,3.0,3.0,1033,high,unspecified,"studio, 1, 2, 3, 4, 5 br",463.0,https://www.99.co/singapore/condos-apartments/...,1.380281,103.943878,0,pasir ris west,pasir ris
2,995264,1 bed condo for sale in icon,10 gopeng street,icon,condo,99-year leasehold,2007.0,1.0,1.0,570,NaN,fully,"studio, 1, 2, 3 br",646.0,https://www.99.co/singapore/condos-apartments/...,1.294668,103.850074,0,bras basah,museum
3,477435,hdb flat for sale in 812b choa chu kang avenue 7,bukit batok / bukit panjang / choa chu kang (d23),keat hong colours,Hdb,99-year leasehold,2017.0,3.0,2.0,1216,NaN,unspecified,"1, 2, 3, 4, 5 br",968.0,https://www.99.co/singapore/hdb/keat-hong-colo...,1.373120,103.746094,0,keat hong,choa chu kang
4,222529,hdb flat for sale in 204 toa payoh north,balestier / toa payoh (d12),toa payoh spring,hdb 4 rooms,99-year leasehold,1973.0,3.0,2.0,936,NaN,unspecified,"1, 2, 3, 4 br",NaN,https://www.99.co/singapore/hdb/toa-payoh-spri...,1.341468,103.849047,0,braddell,toa payoh
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,289879,5 bed house for sale in paradise island,paradise island,paradise island,bungalow,99-year leasehold,2009.0,5.0,6.0,8000,NaN,unspecified,"4, 5, 6, 7, 8 br",29.0,https://www.99.co/singapore/houses/paradise-is...,1.249534,103.844281,0,sentosa,southern islands
6996,396404,5 bed house for sale in orchid village,vanda drive,orchid village,Semi-Detached House,freehold,2010.0,5.0,7.0,6974,NaN,unspecified,"4, 5, 6, 7, 8 br",NaN,https://www.99.co/singapore/houses/orchid-vill...,1.332492,103.800004,0,hillcrest,bukit timah
6997,620343,3 bed condo for sale in the avenir,8 river valley close,the avenir,Condo,freehold,2025.0,3.0,2.0,1141,NaN,partial,"studio, 1, 2, 3, 4, 5 br",376.0,https://www.99.co/singapore/condos-apartments/...,1.294106,103.836735,0,institution hill,river valley
6998,364805,hdb flat for sale in 31 marine crescent,east coast / marine parade (d15),marine crescent ville,hdb,99-year leasehold,1975.0,3.0,1.0,818,NaN,unspecified,"1, 2, 3, 4 br",NaN,https://www.99.co/singapore/hdb/marine-crescen...,1.303736,103.911438,0,marine parade,marine parade


In [7]:
df_train_aux = generate_auxiliary_data(df_train)
df_train_aux

In [8]:
df_test_aux = generate_auxiliary_data(df_test)
df_test_aux

,lat,lng,dist_meters_to_nearest_mrt,nearest_mrt_code,nearest_mrt_line,dist_meters_to_nearest_mall,nearest_mall_index,dist_meters_to_nearest_commercial_centre,nearest_commercial_centre_index,dist_meters_to_nearest_primary_school,nearest_primary_school_index,dist_meters_to_nearest_secondary_school,nearest_secondary_school_index
0,1.344334,103.878690,261.672594,cc12,cc,985.874655,110,1044.920854,31,341.056419,88,343.615356,71
1,1.380281,103.943878,1001.264290,ew1,ew,281.936655,86,1435.511147,15,1078.101901,93,1396.610341,55
2,1.294668,103.850074,251.350375,cc2,cc,212.685809,7,157.232207,37,343.739518,22,562.723759,109
3,1.373120,103.746094,1367.323437,ns4,ns,501.765017,133,4504.563672,1,702.141804,34,1062.566045,122
4,1.341468,103.849047,273.515978,ns18,ns,1009.165929,21,985.918261,32,418.716281,79,360.715437,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,1.249534,103.844281,2992.831027,ew15,ew,2885.355551,142,4272.197274,0,2946.737990,20,3861.603350,29
6996,1.332492,103.800004,389.150989,dt7,dt,965.822466,156,3081.254261,28,743.925796,126,322.085215,79
6997,1.294106,103.836735,729.462991,ns23,ns,532.653630,17,1527.283142,37,1444.051119,3,866.888397,89
6998,1.303736,103.911438,1920.516380,ew6,ew,729.699795,76,2555.480675,30,147.133722,148,749.160547,25


### Save the results

In [9]:
df_train_aux = df_train_aux.drop(['lat', 'lng'], axis=1)

In [10]:
df_train_aux.describe()

In [11]:
df_train_aux.to_csv("raw_data/train_auxiliary_data.csv")

In [12]:
df_test_aux = df_test_aux.drop(['lat', 'lng'], axis=1)

In [13]:
df_test_aux.describe()

,dist_meters_to_nearest_mrt,dist_meters_to_nearest_mall,nearest_mall_index,dist_meters_to_nearest_commercial_centre,nearest_commercial_centre_index,dist_meters_to_nearest_primary_school,nearest_primary_school_index,dist_meters_to_nearest_secondary_school,nearest_secondary_school_index
count,7000.000000,7000.000000,7000.000000,7000.000000,7000.000000,7000.000000,7000.000000,7000.000000,7000.000000
mean,787.154986,785.993608,83.415286,1924.896385,21.052714,735.360130,75.750000,826.887814,65.829429
std,550.611829,524.112192,44.386761,1070.695689,11.970385,534.888693,53.928687,617.764963,41.862228
min,5.257545,0.000000,2.000000,93.334138,0.000000,0.000000,0.000000,67.393276,0.000000
25%,400.608170,398.022578,51.000000,1173.475276,9.000000,365.638318,22.000000,434.994799,29.000000
50%,656.923572,670.208758,86.000000,1681.479925,26.000000,603.752052,70.000000,675.767381,71.000000
75%,1014.802226,1027.768950,117.000000,2423.052923,30.000000,996.729477,126.000000,1029.960484,100.000000
max,3983.074990,3897.147058,164.000000,5574.454058,37.000000,4009.378093,179.000000,4442.112669,147.000000


In [14]:
df_test_aux.to_csv("raw_data/test_auxiliary_data.csv")